# Using paperXai to get your personal arXiv daily digest

The goal of this notebook is to help you use this package and to understand the different components of the pipeline. Briefly, the pipeline can be decomposed into the following sections which will correspond to the sections of this notebook.

- [Fetching the latest arXiv papers](#fetching-the-latest-arxiv-papers)
- [Embedding predefined user questions and sections](#embedding-predefined-user-questions-and-sections)
- [Semantic retrieval](#semantic-retrieval)
- [Generating an automatic report](#generating-an-automatic-report)
- [Sending the personalized newsletter](#sending-the-personalized-newsletter)

For any comments, feel free to reach out directly (see [here](https://sebastianpartarrieu.github.io/)), or via an issue on the github repository.


In [54]:
# initial imports
%load_ext autoreload
%autoreload 2

from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import openai

import paperxai.constants as constants
import paperxai.credentials as credentials
from paperxai.llms import OpenAI
from paperxai.report.retriever import ReportRetriever
from paperxai.prompt.base import Prompt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
openai.api_key = credentials.OPENAI_API_KEY

## Fetching the latest arXiv papers

See script `scripts/get_arxiv_papers.py` to fetch the documents using the arXiv API.


In [2]:
#!python ../scripts/get_arxiv_papers.py --max_results 1000

In [8]:
df_articles = pd.read_csv(constants.ROOT_DIR + "/data/arxiv/base_papers.csv")
df_new_articles = pd.read_csv(constants.ROOT_DIR + "/data/arxiv/current_papers.csv")

## Embedding predefined user questions and sections


### Embedding articles

Let's embed the different articles we've obtained. We pay little attention here to performance, if you want to run this on a larger dataset, it may be worth batching calls and saving the embeddings in a dedicated array (vector database is definitely not useful at this scale).


In [10]:
openai_model = OpenAI(
    chat_model="gpt-3.5-turbo",
    embedding_model="text-embedding-ada-002",
    temperature=0.0,
    max_tokens=1000,
)

df_new_articles["Embeddings"] = df_new_articles["String_representation"].apply(
    lambda x: openai_model.get_embeddings(x)
)

In [11]:
# df_new_articles.to_csv(
#     constants.ROOT_DIR + "/data/arxiv/current_papers_with_embeddings.csv", index=False
# )

In [12]:
article_embeddings = np.vstack(df_new_articles["Embeddings"].values)
np.save(constants.ROOT_DIR + "/data/arxiv/article_embeddings.npy", article_embeddings)

## Semantic retrieval & Generating an automatic report


In [31]:
prompter = Prompt()
report_retriever = ReportRetriever(
    language_model=openai_model,
    prompter=prompter,
    papers_embedding=article_embeddings,
    df_papers=df_new_articles,
)

In [23]:
print(report_retriever.get_chat_response_to_question("What are the latest developments around large language inference and quantization?"))

The latest developments around large language inference and quantization include the introduction of the QuIP method, which utilizes incoherence processing to improve quantization algorithms and achieve viable results using only two bits per weight (Chee, 2023). Additionally, the concept of multilevel large language models has been proposed, where generic and specific models are linked together to improve each other based on user input and internet information, inspired by the functionality of the human brain (Gong, 2023). Furthermore, a personalized mixed-precision quantization method called myQASR has been developed, which tailors quantization schemes for diverse users in automatic speech recognition models without fine-tuning (Fish, 2023).


In [50]:
report_retriever.create_report()

Getting responses for section: Large Language Model inference optimization
Getting responses for section: Large Language Model training optimization


{'Large Language Model inference optimization': {'questions': ['What are the latest developments around large language inference and quantization?',
   'What are the latest developments around large language inference and Mixture of Experts?',
   'What are the latest developments around large language inference memory optimization?'],
  'chat_responses': ['The latest developments around large language inference and quantization include the introduction of the QuIP method, which utilizes incoherence processing to improve quantization algorithms for large language models (Chee, 2023), the proposal of multilevel large language models that unify generic and specific models to improve performance based on user input and internet information (Gong, 2023), and the development of myQASR, a personalized mixed-precision quantization method for automatic speech recognition models that tailors quantization schemes for diverse users under any memory requirement (Fish, 2023).',
   'The latest develo

In [61]:
list(report_retriever.report.items())

[('Large Language Model inference optimization',
  {'questions': ['What are the latest developments around large language inference and quantization?',
    'What are the latest developments around large language inference and Mixture of Experts?',
    'What are the latest developments around large language inference memory optimization?'],
   'chat_responses': ['The latest developments around large language inference and quantization include the introduction of the QuIP method, which utilizes incoherence processing to improve quantization algorithms for large language models (Chee, 2023), the proposal of multilevel large language models that unify generic and specific models to improve performance based on user input and internet information (Gong, 2023), and the development of myQASR, a personalized mixed-precision quantization method for automatic speech recognition models that tailors quantization schemes for diverse users under any memory requirement (Fish, 2023).',
    'The latest

In [62]:
report_retriever.print_report()

Section: Large Language Model inference optimization
Question: What are the latest developments around large language inference and quantization?
LLM response: The latest developments around large language inference and quantization include the introduction of the QuIP method, which utilizes incoherence processing to improve quantization algorithms for large language models (Chee, 2023), the proposal of multilevel large language models that unify generic and specific models to improve performance based on user input and internet information (Gong, 2023), and the development of myQASR, a personalized mixed-precision quantization method for automatic speech recognition models that tailors quantization schemes for diverse users under any memory requirement (Fish, 2023).
Question: What are the latest developments around large language inference and Mixture of Experts?
LLM response: The latest developments around large language inference and Mixture of Experts include the unification of gen

## Sending the personalized newsletter
